In [1]:
######## CLASSIFICATION OF SENTENCESS BASED ON Universal-sentence-encoder and keras modelling #########
## https://www.dlology.com/blog/keras-meets-universal-sentence-encoder-transfer-learning-for-text-data/

In [458]:
import tensorflow as tf
import tensorflow_hub as hub
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda
from keras.models import Model
import numpy as np

In [459]:
def get_dataframe(filename):
    lines = open(filename, 'r').read().splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i].split(' ')[0]
        label = label.split(",")[0]
        text = ' '.join(lines[i].split(',')[1:])
        text = re.sub(',','', text)
        data.append([label, text])

    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

df_train = get_dataframe('sentences.txt')
df_train.head()

,label,text
0,0,I need daily rations in Attur
1,0,Give rice dal oil
2,0,people are hungary
3,0,provide rations for 6 people for 10 days
4,0,Need prepared food for 20 people in slum


In [460]:
# df_train

In [461]:
### list of one-hot encoded labels
train_text = df_train['text'].tolist()
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

train_label = np.asarray(pd.get_dummies(df_train.label), dtype = np.int8)

In [462]:
category_counts = len(df_train.label.cat.categories)
module_url = "tf_sent_encoder_2_MODEL_PRETRAINED" 
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

In [340]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]


In [8]:

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
dense1 = Dense(256, activation='relu')(embedding)
dense2 = Dense(128, activation='relu')(dense1)
pred = Dense(category_counts, activation='softmax')(dense2)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

W0504 14:34:07.140520 20892 deprecation_wrapper.py:119] From c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0504 14:34:07.143542 20892 deprecation_wrapper.py:119] From c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0504 14:34:09.596715 20892 deprecation_wrapper.py:119] From c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0504 14:34:09.793557 20892 deprecation_wrapper.py:119] From c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 645       
Total params: 164,869
Trainable params: 164,869
Non-trainable params: 0
_________________________________________________________________


In [388]:
df_test = get_dataframe('test.txt')
### list of one-hot encoded labels
test_text = df_test['text'].tolist()
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

test_label = np.asarray(pd.get_dummies(df_test.label), dtype = np.int8)

In [390]:
test_text

array([['Need food for diabeties patient'],
       ['I want bread and rice for my home'],
       ['arm broken need to go to hospital'],
       ['food poisoning by eating old distributed food'],
       ['network tower broken down in whitefield'],
       ['calls are not connecting from airtel'],
       ['I am a doctor  my car brokedown and want it to get repaired urgently.'],
       ['My visa about to get expire. Want some travel arrangements to be done to fly back to Austia.'],
       ['My neighbor attempted suicide .She is in hospital right now  urgent help needed'],
       ['I saw bruises on my maid face. I think she is a victim of domestic violence.']],
      dtype=object)

In [21]:
from keras import backend as K

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),
            epochs=10,batch_size=20)
  model.save_weights('./model.h5')

Train on 40 samples, validate on 10 samples
Epoch 1/10
20/40 [==============>...............] - ETA: 2s - loss: 1.6075 - acc: 0.2000

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000001BD7E89D828>
Traceback (most recent call last):
  File "c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


40/40 [==============================] - 4s 88ms/step - loss: 1.6110 - acc: 0.1500 - val_loss: 1.5837 - val_acc: 0.4000
Epoch 2/10
40/40 [==============================] - 0s 2ms/step - loss: 1.5484 - acc: 0.7250 - val_loss: 1.5432 - val_acc: 0.6000
Epoch 3/10
40/40 [==============================] - 0s 2ms/step - loss: 1.4864 - acc: 0.9500 - val_loss: 1.4984 - val_acc: 0.7000
Epoch 4/10
20/40 [==============>...............] - ETA: 0s - loss: 1.4171 - acc: 0.9500

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000001BD7E89D828>
Traceback (most recent call last):
  File "c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


40/40 [==============================] - 0s 2ms/step - loss: 1.4202 - acc: 0.9750 - val_loss: 1.4440 - val_acc: 0.7000
Epoch 5/10
40/40 [==============================] - 0s 1ms/step - loss: 1.3395 - acc: 0.9750 - val_loss: 1.3813 - val_acc: 0.7000
Epoch 6/10
40/40 [==============================] - 0s 2ms/step - loss: 1.2457 - acc: 1.0000 - val_loss: 1.3108 - val_acc: 0.7000
Epoch 7/10
40/40 [==============================] - 0s 2ms/step - loss: 1.1359 - acc: 1.0000 - val_loss: 1.2333 - val_acc: 0.7000
Epoch 8/10
40/40 [==============================] - 0s 2ms/step - loss: 1.0181 - acc: 1.0000 - val_loss: 1.1511 - val_acc: 0.7000
Epoch 9/10
40/40 [==============================] - 0s 1ms/step - loss: 0.8930 - acc: 1.0000 - val_loss: 1.0709 - val_acc: 0.7000
Epoch 10/10
40/40 [==============================] - 0s 2ms/step - loss: 0.7621 - acc: 1.0000 - val_loss: 0.9899 - val_acc: 0.7000


In [22]:
new_text = ["wheat exhausted in hospital canteen", 
            "my neighbors house has crying voices. I think they need help"]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text)

categories = df_train.label.cat.categories.tolist()
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
print(predict_labels)

Exception ignored in: <function BaseSession._Callable.__del__ at 0x000001BD7E89D828>
Traceback (most recent call last):
  File "c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\client\session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


['0', '4']


In [23]:
predicts

array([[0.4154314 , 0.2754173 , 0.07836786, 0.13586688, 0.09491654],
       [0.16206545, 0.1603227 , 0.15349114, 0.09350415, 0.43061662]],
      dtype=float32)

In [118]:
###################################################################################################

In [ ]:
############ With LARGE datasets ##############

In [474]:
import tensorflow as tf
import tensorflow_hub as hub
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda
from keras.models import Model
import numpy as np

In [609]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd

df = pd.DataFrame(columns=['label','text'])
trainset = sklearn.datasets.load_files(container_path = 'datalarge',encoding = 'UTF-8')

def get_dataframe(lines):
#     lines = filename.splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i][0]
    #     label = label.split(",")[0]
        text = ' '.join(lines[i][1:])
        text = re.sub(',','', text)
        data.append([label, text])
    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

for k in range(len(trainset.target)):
    labelled_sent= trainset.data[k].strip().split('\r\n')
    Label=[trainset.target[k]]*len(labelled_sent)
    labelled_emb = list(zip([s for s in Label], [s for s in labelled_sent]))
    df_train = get_dataframe(labelled_emb)
    df=pd.concat([df,df_train])

In [579]:
X_train, X_test, y_train, y_test = train_test_split(df, df['label'], test_size=0.2,random_state=42)
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)


# X_train.reset_index(drop=True)
# X_test.reset_index(drop=True)
# y_train.reset_index(drop=True)
# y_test.reset_index(drop=True)

In [477]:
train_text = np.array(X_train, dtype=object)[:,1][:,np.newaxis]
train_label = np.asarray(pd.get_dummies(X_train.label), dtype = np.int8)

In [478]:
test_text = np.array(X_test, dtype=object)[:,1][:,np.newaxis]
test_label = np.asarray(pd.get_dummies(X_test.label), dtype = np.int8)

In [481]:
category_counts = len(np.unique(X_train.label))
module_url = "tf_sent_encoder_2_MODEL_PRETRAINED" 
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

In [482]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]


In [483]:

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
dense1 = Dense(256, activation='relu')(embedding)
dense2 = Dense(128, activation='relu')(dense1)
pred = Dense(category_counts, activation='softmax')(dense2)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [486]:
from keras import backend as K

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label),epochs=5,batch_size=50)
  model.save_weights('./model.h5')

Train on 3540 samples, validate on 885 samples
Epoch 1/5
3540/3540 [==============================] - 264s 75ms/step - loss: 0.8755 - acc: 0.6969 - val_loss: 0.2986 - val_acc: 0.9051
Epoch 2/5
3540/3540 [==============================] - 15s 4ms/step - loss: 0.2148 - acc: 0.9325 - val_loss: 0.1674 - val_acc: 0.9401
Epoch 3/5
3540/3540 [==============================] - 15s 4ms/step - loss: 0.1162 - acc: 0.9664 - val_loss: 0.1158 - val_acc: 0.9616
Epoch 4/5
3540/3540 [==============================] - 15s 4ms/step - loss: 0.0733 - acc: 0.9828 - val_loss: 0.0951 - val_acc: 0.9661
Epoch 5/5
3540/3540 [==============================] - 14s 4ms/step - loss: 0.0484 - acc: 0.9898 - val_loss: 0.0868 - val_acc: 0.9661


In [511]:
new_text = ["wheat exhausted in hospital canteen", 
            "my neighbors house has crying voices. I think they need help"]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text)


In [541]:

categories = np.unique(X_train.label)
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
dict=['abuse','communication','food','fuel','health','travel']
ANS=[dict[s] for s in predict_labels]
print(ANS)

['food', 'abuse']


In [544]:
np.around(predicts*100,decimals=1)

array([[ 0. ,  0. , 58.2,  0. , 41.8,  0. ],
       [86.8, 11. ,  0. ,  0.2,  1.8,  0. ]], dtype=float32)

In [ ]:
dict=['abuse','communication','food','fuel','health','travel']

In [ ]:
############ Combining both above cases ################3

In [547]:
new_text = ["wheat exhausted in hospital canteen and my neighbors house has crying voices. I think they need help",
           "GIVE ME OIL"]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights('./model.h5')  
  predicts = model.predict(new_text)


In [548]:

categories = np.unique(X_train.label)
predict_logits = predicts.argmax(axis=1)
predict_labels = [categories[logit] for logit in predict_logits]
dict=['abuse','communication','food','fuel','health','travel']
ANS=[dict[s] for s in predict_labels]
print(ANS)

['abuse', 'fuel']


In [549]:
np.around(predicts*100,decimals=1)

array([[63. ,  1.2,  6.9,  8.5, 18.8,  1.6],
       [ 0. ,  0. ,  0.4, 99.5,  0.1,  0. ]], dtype=float32)

In [ ]:
####################### TRY THIS - ANUP ####################################

In [ ]:
#################################################################

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
# import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from tensorflow import keras
from keras.layers import Input, Embedding, LSTM, Dense, Lambda
from keras.models import Model
import numpy as np

c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\i511977\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\i511

In [14]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd

df = pd.DataFrame(columns=['label','text'])
trainset = sklearn.datasets.load_files(container_path = 'datalarge',encoding = 'UTF-8')

def get_dataframe(lines):
#     lines = filename.splitlines()
    data = []
    for i in range(0, len(lines)):
        label = lines[i][0]
    #     label = label.split(",")[0]
        text = ' '.join(lines[i][1:])
        text = re.sub(',','', text)
        data.append([label, text])
    df = pd.DataFrame(data, columns=['label', 'text'])
    df.label = df.label.astype('category')
    return df

for k in range(len(trainset.target)):
    labelled_sent= trainset.data[k].strip().split('\r\n')
    Label=[trainset.target[k]]*len(labelled_sent)
    labelled_emb = list(zip([s for s in Label], [s for s in labelled_sent]))
    df_train = get_dataframe(labelled_emb)
    df=pd.concat([df,df_train])
    df.reset_index(inplace=True, drop=True)

In [15]:
df=df[:40]

In [16]:
import random
data=[]
length=len(df)
for i in range(length):
    num1=random.randrange(length)
    num2=random.randrange(length)
    data.append([df.label[num1],df.label[num2],df.text[num1] + df.text[num2]])
    
df2 = pd.DataFrame(data, columns=['label1','label2', 'text'])


In [17]:
X_train, X_test, y_train, y_test = train_test_split(df2, df2['label1'], test_size=0.2)
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)


# X_train.reset_index(drop=True)
# X_test.reset_index(drop=True)
# y_train.reset_index(drop=True)
# y_test.reset_index(drop=True)

In [18]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

main=[]
for i in range(len(X_train)):
    arr=np.array([X_train['label1'][i],X_train['label2'][i]])
    main.append(arr)
    
# Create MultiLabelBinarizer object
one_hot = MultiLabelBinarizer()

# One-hot encode data
train_text = np.array(X_train, dtype=object)[:,1][:,np.newaxis]
train_label =np.asarray(one_hot.fit_transform(main), dtype = np.int8)

In [19]:
main=[]
for i in range(len(X_test)):
    arr=np.array([X_test['label1'][i],X_test['label2'][i]])
    main.append(arr)
    
# Create MultiLabelBinarizer object
one_hot = MultiLabelBinarizer()

test_text = np.array(X_test, dtype=object)[:,1][:,np.newaxis]
test_label = np.asarray(one_hot.fit_transform(main), dtype = np.int8)



In [20]:
category_counts = len(np.unique(df.label))
module_url = "tf_sent_encoder_2_MODEL_PRETRAINED" 
embed = hub.Module(module_url)
embed_size = embed.get_output_info_dict()['default'].get_shape()[1].value

In [21]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)),signature="default", as_dict=True)["default"]

# def sentence_encoder(input_text):
#     return embed(tf.squeeze(input_text))

In [22]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding,output_shape=(embed_size,))(input_text)
dense1 = Dense(1024, activation='relu')(embedding)
dense2 = Dense(128, activation='relu')(dense1)
pred = Dense(category_counts, activation='sigmoid')(dense2)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [23]:
# from keras import backend as K

# with tf.Session() as session:
#   K.set_session(session)
#   session.run(tf.global_variables_initializer())
#   session.run(tf.tables_initializer())
#   history = model.fit(train_text, 
#             train_label,
#             validation_data=(test_text, test_label),epochs=1)
#   model.save_weights('./model.h5')


from keras import backend as K

with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  history = model.fit(train_text, 
            train_label,
            validation_data=(test_text, test_label))
  model.save_weights('./model1.h5')

Train on 16 samples, validate on 4 samples
Epoch 1/1


InternalError: Unable to get element as bytes.

In [ ]:
new_text = ["No network connectivity in my area for ordering groceries. Need ration urgently",
"Need gas cylinder as my mobile broke down.",
"My father is sick. i want daily ration",
"Yesterday i saw few people bullying a person from my locality. I think he needs help.i need cab as my car broke down to go to hospital."]

new_text = np.array(new_text, dtype=object)[:, np.newaxis]
predicts = model.predict(new_text)
dictt=['abuse','communication','food','fuel','health','travel']

for j in range(len(new_text)):
    print("QUERY "+str(j) +'-> ' + new_text[j])
    print('\n')
    for i in range(len(dictt)):
        print('\t'+dictt[i]+":"+str(np.around(predicts*100,decimals=1)[j][i]))
    print('\n')